<a href="https://colab.research.google.com/github/yujinc129-oss/Drama_data_project/blob/main/notebooks/04_encoding_%26_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyGithub

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn. ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

import os
import pandas as pd
from google.colab import userdata
from github import Github, GithubException

In [20]:
drama =pd.read_json('https://github.com/yujinc129-oss/Drama_data_project/raw/refs/heads/main/data/processed/03_feature', orient="records")

In [21]:
drama.rename(columns={'start airing_x' : 'start airing','start airing_y':'season'},inplace=True)

#인코딩

In [22]:
mlb = MultiLabelBinarizer()

fmt = mlb.fit_transform(drama['genres'])
gen = pd.DataFrame(fmt, columns=mlb.classes_, index=drama.index)


mlb = MultiLabelBinarizer()

fmt = mlb.fit_transform(drama['day'])
day = pd.DataFrame(fmt, columns=mlb.classes_, index=drama.index)

mlb = MultiLabelBinarizer()

fmt = mlb.fit_transform(drama['network'])
net = pd.DataFrame(fmt, columns=mlb.classes_, index=drama.index)


In [23]:
net

,CJ,JPP,JTBC,KBS,MBC,NETFLIX,OCN,SBS,TVN,WEB
0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,1,1,0,0,0,0
4,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2172,0,0,0,1,0,0,0,0,0,0
2173,0,0,0,0,1,0,0,0,0,0
2174,1,0,0,0,0,0,0,0,0,0
2175,0,0,0,0,1,0,0,0,0,0


In [24]:
others = pd.get_dummies(drama[['role','gender']])

In [25]:
X = pd.concat([drama['age'],gen, day, net, others], axis=1)


In [26]:
X

,age,action,comedy,drama,etc,food,hist_war,music,romance,sf,...,MBC,NETFLIX,OCN,SBS,TVN,WEB,role_Main Role,role_Support Role,gender_Female,gender_Male
0,34,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,True,False,False,True
1,34,1,0,1,0,0,1,0,0,0,...,0,1,0,0,0,0,True,False,False,True
2,33,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,True,False,False,True
3,32,0,0,1,0,0,0,0,1,0,...,1,1,0,0,0,0,True,False,False,True
4,31,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2172,40,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,False,True,False,True
2173,32,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,True,False,False,True
2174,27,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,True,False,False,True
2175,32,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,True,False,False,True


In [28]:
X_data = X.copy()
data_processed = pd.concat([X_data, drama[['start airing','main_ratio','platform_tier','season']]], axis=1)

In [29]:
#X변수
X = data_processed
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2177 entries, 0 to 2176
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                2177 non-null   int64  
 1   action             2177 non-null   int64  
 2   comedy             2177 non-null   int64  
 3   drama              2177 non-null   int64  
 4   etc                2177 non-null   int64  
 5   food               2177 non-null   int64  
 6   hist_war           2177 non-null   int64  
 7   music              2177 non-null   int64  
 8   romance            2177 non-null   int64  
 9   sf                 2177 non-null   int64  
 10  society            2177 non-null   int64  
 11  sports             2177 non-null   int64  
 12  thriller           2177 non-null   int64  
 13  youth              2177 non-null   int64  
 14  friday             2177 non-null   int64  
 15  monday             2177 non-null   int64  
 16  saturday           2177 

In [30]:
#y 변수
y= drama['score']

In [31]:
X.to_json('X', orient='records')
y.to_json('y', orient='records')

# [Raw] 버튼을 눌러 나온 URL을 여기에 붙여넣으세요.
!wget -O git_push.py "https://github.com/yujinc129-oss/Drama_data_project/raw/refs/heads/main/utils/git_push.py"

from git_push import git_push


--2025-08-06 05:42:55--  https://github.com/yujinc129-oss/Drama_data_project/raw/refs/heads/main/utils/git_push.py
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yujinc129-oss/Drama_data_project/refs/heads/main/utils/git_push.py [following]
--2025-08-06 05:42:56--  https://raw.githubusercontent.com/yujinc129-oss/Drama_data_project/refs/heads/main/utils/git_push.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2582 (2.5K) [text/plain]
Saving to: ‘git_push.py’

git_push.py         100%[===================>]   2.52K  --.-KB/s    in 0s      

2025-08-06 05:42:56 (22.8 MB/s) - ‘git_push.py’ saved 

In [32]:

git_push('/content/X')
git_push('/content/y')

✅ 파일 업데이트 성공: 'data/processed/X'
✅ 파일 업데이트 성공: 'data/processed/y'
